# Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

tf.random.set_seed(42)
np.random.seed(42)

# EDA

In [ ]:
df = pd.read_csv("/kaggle/input/lstm/AP003.csv")
df.head(5)

In [ ]:
df.info()

In [ ]:
num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(include="object").columns

In [ ]:
df['From Date'] = pd.to_datetime(df['From Date'])
df = df.set_index('From Date')

In [ ]:
df.isna().sum()

In [ ]:
df[num_cols] = df[num_cols].interpolate(method='time').fillna(method='ffill').fillna(method='bfill')

In [ ]:
df.isna().sum().sum()

In [ ]:
df = df.drop('To Date', axis = 1)

In [ ]:
df[df.duplicated()].sum()

In [ ]:
df_daily = df[num_cols].resample('D').mean()

axes = df_daily.plot(subplots=True, figsize=(14, 2 * len(num_cols)))
plt.suptitle('Daily Trends (2017–2023)', fontsize=16)
plt.tight_layout()

for ax in axes:
    ax.legend(loc='upper right')


In [ ]:
data = df[df.index >= '2021-01-01'].resample('D').mean()

year_bounds = pd.date_range(start='2021-01-01', end=data.index.max(), freq='YS')

fig, axes = plt.subplots(len(num_cols), 1, figsize=(14, 3 * len(num_cols)), sharex=False)

for i, feature in enumerate(num_cols):
    ax = axes[i]
    data[feature].plot(ax=ax)
    ax.set_title(f'Daily Avg of {feature} (2021–{data.index.max().year})')
    ax.set_ylabel(feature)

    for year_start in year_bounds:
        ax.axvline(year_start, color='red', linestyle='--', linewidth=1)

    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.tick_params(axis='x')

plt.tight_layout()
plt.show()


In [ ]:
corr = df[num_cols].corr()

mask = np.triu(np.ones_like(corr, dtype=bool), k=1)

plt.figure(figsize=(12, 10))
sns.heatmap(corr, mask=mask, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title('Correlation of each variables', fontsize=12)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Preprocessing

In [ ]:
df_model = pd.read_csv('/kaggle/input/uas-dl-lstm/AP003.csv')
df_model

In [ ]:
df_model.set_index('From Date', inplace=True)
df_model = df_model.drop(['To Date', 'VWS (m/s)', 'Xylene (ug/m3)', 'RF (mm)', 'NH3 (ug/m3)', 'SO2 (ug/m3)', 'Temp (degree C)','BP (mmHg)'], axis = 1)
df_model.head(5)

## Splitting the data

In [ ]:
n_samples = len(df_model)
train_end = int(n_samples * 0.8)
val_end = int(n_samples * 0.9)

In [ ]:
train = df_model.iloc[:train_end].copy()
val = df_model.iloc[train_end:val_end].copy()
test = df_model.iloc[val_end:].copy()

In [ ]:
print(f"Training set: {len(train)}")
print(f"Validation set: {len(val)}")
print(f"Test set: {len(test)}")

## Filling NaN

In [ ]:
train_clean = train.interpolate(method='linear', limit_direction='both')
train_clean = train_clean.fillna(method='ffill').fillna(method='bfill')

val_clean = val.interpolate(method='linear', limit_direction='both')
val_clean = val_clean.fillna(method='ffill').fillna(method='bfill')

test_clean = test.interpolate(method='linear', limit_direction='both')
test_clean = test_clean.fillna(method='ffill').fillna(method='bfill')

In [ ]:
col_name_x = train_clean.columns

train_idx = train_clean.index
test_idx = test_clean.index
val_idx = val_clean.index

## Scaling

In [ ]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_clean)
val_scaled = scaler.transform(val_clean)
test_scaled = scaler.transform(test_clean)

In [ ]:
train_scaled_df = pd.DataFrame(train_scaled, columns=col_name_x, index=train_idx)
val_scaled_df = pd.DataFrame(val_scaled, columns=col_name_x, index=val_idx)
test_scaled_df = pd.DataFrame(test_scaled, columns=col_name_x, index=test_idx)

In [ ]:
target_column = 'AT (degree C)'
lookback = 5
forecast = 1

## Sequencing

In [ ]:
X_train, y_train = [], []
target_idx = col_name_x.get_loc(target_column)

for i in range(lookback, len(train_scaled_df) - forecast + 1):
    X_train.append(train_scaled_df.iloc[i-lookback:i].values)
    y_train.append(train_scaled_df.iloc[i + forecast - 1, target_idx])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_val, y_val = [], []

for i in range(lookback, len(val_scaled_df) - forecast + 1):
    X_val.append(val_scaled_df.iloc[i-lookback:i].values)
    y_val.append(val_scaled_df.iloc[i + forecast - 1, target_idx])

X_val = np.array(X_val)
y_val = np.array(y_val)

In [ ]:
X_test, y_test = [], []

for i in range(lookback, len(test_scaled_df) - forecast + 1):
    X_test.append(test_scaled_df.iloc[i-lookback:i].values)
    y_test.append(test_scaled_df.iloc[i + forecast - 1, target_idx])

X_test = np.array(X_test)
y_test = np.array(y_test)

# Modelling

## Base Model

In [ ]:
base_model = Sequential([
    LSTM(10, input_shape=(lookback, len(col_name_x)), return_sequences=False),
    Dense(1, activation = 'linear')
])

In [ ]:
base_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

In [ ]:
history_base = base_model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)


In [ ]:
y_pred_base = base_model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred_base)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_base)
r2 = r2_score(y_test, y_pred_base)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

In [ ]:
train_loss = history_base.history['loss']
val_loss = history_base.history['val_loss']
plt.plot(train_loss,label="Train Loss")
plt.plot(val_loss,label='Val Loss')
plt.title("Gradient Descent")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
time_index = pd.to_datetime(test_scaled_df.index[lookback + forecast - 1:])
time_index = time_index[:len(y_test)]

y_pred = base_model.predict(X_test).squeeze()

df_result = pd.DataFrame({
    "Time": time_index,
    "Actual": y_test,
    "Predicted": y_pred
})

df_result.set_index("Time", inplace=True)

df_daily = df_result.resample("D").mean()

plt.figure(figsize=(12, 5))
plt.plot(df_daily.index, df_daily["Actual"], label="Daily Avg Actual AT", linewidth=2)
plt.plot(df_daily.index, df_daily["Predicted"], label="Daily Avg Predicted AT", linewidth=2)

plt.title("Daily Average AT (degree C): Actual vs Predicted")
plt.xlabel("Date")
plt.ylabel("AT (degree C)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Tuned Model

In [ ]:
modified_model = Sequential([

    LSTM(10, return_sequences=True, recurrent_dropout=0.2),
    BatchNormalization(),
    LSTM(5, return_sequences= False),
    BatchNormalization(),
    
    Dense(1, activation='linear')
])

In [ ]:
modified_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

In [ ]:
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=8,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
    filepath='/kaggle/working/.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='max',
    save_weights_only=False,
    verbose=1
  )
]

In [ ]:
history_modified = modified_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
y_pred_modified = modified_model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred_modified)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_modified)
r2 = r2_score(y_test, y_pred_modified)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

In [ ]:
train_loss = history_modified.history['loss']
val_loss = history_modified.history['val_loss']
plt.plot(train_loss,label="Train Loss")
plt.plot(val_loss,label='Val Loss')
plt.title("Gradient Descent")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
time_index = pd.to_datetime(test_scaled_df.index[lookback + forecast - 1:])
time_index = time_index[:len(y_test)]

y_pred = modified_model.predict(X_test).squeeze()

df_result = pd.DataFrame({
    "Time": time_index,
    "Actual": y_test,
    "Predicted": y_pred
})

df_result.set_index("Time", inplace=True)

df_daily = df_result.resample("D").mean()

plt.figure(figsize=(12, 5))
plt.plot(df_daily.index, df_daily["Actual"], label="Daily Avg Actual AT", linewidth=2)
plt.plot(df_daily.index, df_daily["Predicted"], label="Daily Avg Predicted AT", linewidth=2)

plt.title("Daily Average AT (degree C): Actual vs Predicted")
plt.xlabel("Date")
plt.ylabel("AT (degree C)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Kesimpulan

Interpretasi: Dari Hasil kedua model diatas dapat kita simpulkan model modified tidak memiliki perbedaan kemampuan prediksi yang jauh bahkan bisa dibilang sama. Dari graph gradient descent kita dapat melihat bahwa model tidak dibilang overfit / underfit karena loss dari train dan validation sudah sangat dekat sehingga model ini cocok untuk melakukan forecasting. Kita juga dapat melihat perbandingan predicted data dan actual data pada line chart dimana predicted dan actual sudah sangat mendekati

# Bonus

Problem untuk Case sekarang bukanlah multivariate karena kita hanya menggunakan model LSTM untuk memprediksi satu fitur yaitu A, Problem ini bisa menjadi multivariate jika kita melakukan prediksi dengan banyak output seperti misalnya kita juga ingin memprediksi AT, CO, dan CO2 pada 1 jam kedepannya, jika seperti itu maka kita akan menggunakan multivariate time series.

Menentukan relevansi fitur bisa menggunakan tes korelasi dengan menentukan p-value untuk mengetahui fitur apa saja yang penting, mencari korelasi seperti diatas dengan method pearson karena distribusi data kita anggap normal, dan tentunya butuh expert dalam topik tersebut.

Secara simpelnya kita hanya menggunakan multivariate time series saat ingin memprediksi lebih dari satu fitur dimana pada case ini jika fitur-fitur tersebut relevan dan saling berhubungan maka kita dapat memprediksi banyak fitur dengan multivariate time series.

